In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile

import sys
sys.path.append('/home/azureuser/cloudfiles/code/Users/cl43/torchcrepe')
import torchcrepe
from torchcrepe.predict_custom import predict as predict_custom
from torchcrepe.predict_custom import load_audio

## Load audio

In [2]:
data_path = 'data/test/CSD/wav/converted'
pitch_path = 'out/pitch'
num_files = 60 # How many data points to evaluate on, including skipped files? (Set to -1 to evaluate on all available files)
skip_files = ['en002a', 'en002b', 'en004a', 'en005a', 'en005b', 'en007a', 'en007b', 'en008a', 'en008b',  # 6
                'en009a', 'en009b', 'en010a', 'en010b', 'en011a', 'en012a', 'en012b', 'en013b', 'en014a', 'en014b', # 2
                'en015a', 'en016a', 'en018a', 'en018b', 'en019b', 'en020a', 'en020b', 'en021a', 'en022a', 'en022b', 'en023a', 'en023b', 'en024a', # 7
                'en025a', 'en025b', 'en026a', 'en026b', 'en027a', 'en027b' # 6 (en028 and en029 and en030)
                ] 

model = 'kl_full'
capacity = 'full'

# e.g. out/pitch/base_tiny
full_pitch_path = os.path.join(pitch_path, model)

# Read in wavfiles
raw_filenames = []
wavfiles = [] # (sample rate, audio data)
i = 0
for filename in os.listdir(data_path):
    if filename.endswith(".wav"):
        i += 1
        # limit # files
        if i > num_files and num_files > 0:
            break

        # Skip undesired files
        if filename[:-4] in skip_files:
            continue

        # Skip files that were already done
        raw_filename = filename[:-4]
        saved_filename = os.path.join(full_pitch_path, raw_filename + '.pkl')
        if os.path.exists(saved_filename):
            print(saved_filename, 'already exists')
            continue
        
        # print(os.path.join(data_path, filename))
        raw_filenames.append(raw_filename)
        audio, sr = load_audio(os.path.join(data_path, filename))
        wavfiles.append((audio, sr))
        # print(wavfiles[-1])


## Run predictions

In [3]:
# Run/load crepe predictions
def save_obj(filename, objs):
    with open(filename, 'wb') as outp:
        for obj in objs:
            pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

def read_obj(filename, num_to_read):
    objs = []
    with open(filename, 'rb') as inp:
        for i in range(num_to_read):
            objs.append(pickle.load(inp))
        return objs

times = []
frequencies = []
confidences = []
activations = []
# Run/load crepe predictions
for i, file_info in enumerate(wavfiles):
    filename = os.path.join(full_pitch_path, raw_filenames[i] + '.pkl')
    print('audio shape:', file_info[0].shape)
    if os.path.exists(filename):
        # Read cached prediction outputs
        arr = read_obj(filename, 4)
        frequency = arr[1]
        confidence = arr[2]
        activation = arr[3]
        print(filename, 'already exists')
    else:
        # Run prediction and save output
        sr = file_info[1]
        audio = file_info[0]
        # print(sr, audio)
        # print(audio.detach().cpu().numpy().squeeze(0))
        # print(audio)
        # hop length 20ms
        time, frequency, confidence, activation = predict_custom(audio, sr, hop_length=sr/50, model=model, decoder=torchcrepe.decode.weighted_argmax, capacity=capacity, special='KL')
        save_obj(filename, [time, frequency, confidence, activation]) # Uncomment to save predictions 
        print('saved to', filename)
    
    # # freq = 27.5 * 2 ** ((msg.note - 21) / 12) 
    # # Convert frequency back to note -> note = 12 * log_2(freq / 27.5) + 21
    # if not use_frequency:
    #     for idx in range(len(frequency)):
    #         frequency[idx] = 12 * np.log2(frequency[idx] / 27.5) + 21
    #         if not frequency[idx] >= 0:
    #             print(frequency[idx])

    # print(frequency, confidence, activation)
    # print(frequency.shape)
    # import torch
    # print(torch.mean(frequency))

audio shape: torch.Size([1, 2857680])
saved to out/pitch/kl_full/en001a.pkl
audio shape: torch.Size([1, 2857680])
saved to out/pitch/kl_full/en001b.pkl
audio shape: torch.Size([1, 2761044])
saved to out/pitch/kl_full/en003a.pkl
audio shape: torch.Size([1, 2761044])
saved to out/pitch/kl_full/en003b.pkl
audio shape: torch.Size([1, 2822400])
saved to out/pitch/kl_full/en004b.pkl
audio shape: torch.Size([1, 3880801])
saved to out/pitch/kl_full/en006a.pkl
audio shape: torch.Size([1, 3880801])
saved to out/pitch/kl_full/en006b.pkl
audio shape: torch.Size([1, 4795875])
saved to out/pitch/kl_full/en011b.pkl
audio shape: torch.Size([1, 1984500])
saved to out/pitch/kl_full/en013a.pkl
audio shape: torch.Size([1, 3024000])
saved to out/pitch/kl_full/en015b.pkl
audio shape: torch.Size([1, 3112941])
saved to out/pitch/kl_full/en016b.pkl
audio shape: torch.Size([1, 840001])
saved to out/pitch/kl_full/en017a.pkl
audio shape: torch.Size([1, 840001])
saved to out/pitch/kl_full/en017b.pkl
audio shape: t

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
